
# Fetching atlases and annotations

This example demonstrates how to use :mod:`neuromaps.datasets` to fetch
atlases and annotations.


Much of the functionality of the ``neuromaps`` toolbox relies on the
atlases and atlas files provided with it. In many cases these atlases are
fetched "behind-the-scenes" when you call functions that depend on them, but
they can be accessed directly.

There is a general purpose :func:`neuromaps.datasets.fetch_atlas`
function that can fetch any of the atlases provided with ``neuromaps``:



In [1]:
!pip install git+https://github.com/netneurolab/neuromaps.git
from neuromaps import datasets

fslr = datasets.fetch_atlas(atlas='fslr', density='32k')
print(fslr.keys())

  Cloning https://github.com/netneurolab/neuromaps.git to /private/var/folders/t2/8t74_04d3t5gx4vjqq895zbh0000gn/T/pip-req-build-_rkj09bd
  Running command git clone --filter=blob:none --quiet https://github.com/netneurolab/neuromaps.git /private/var/folders/t2/8t74_04d3t5gx4vjqq895zbh0000gn/T/pip-req-build-_rkj09bd
  Resolved https://github.com/netneurolab/neuromaps.git to commit f0ed67c44d633061b93c6d337a1b233f6895c408
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


[fetch_files] Downloading data from 
https://files.osf.io/v1/resources/4mw3a/providers/osfstorage/60b684b53a6df1021bd4df2d ...

[fetch_files]  ...done. (3 seconds, 0 min)

[fetch_files] Extracting data from 
/Users/kevinhuang/neuromaps-data/599046a594e0e45c04e90291c2348cbe/fsLR32k.tar.gz...

[fetch_files] .. done.

dict_keys(['midthickness', 'inflated', 'veryinflated', 'sphere', 'medial', 'sulc', 'vaavg'])


The values corresponding to the keys of the atlas dictionary are length-2
lists containing filepaths to the downloaded data. All surface atlas files
are provide in gifti format (whereas MNI files are in gzipped nifti format).

You can load them directly with ``nibabel`` to confirm their validity:



In [2]:
import nibabel as nib
lsphere, rsphere = fslr['sphere']
lvert, ltri = nib.load(lsphere).agg_data()
print(lvert.shape, ltri.shape)

(32492, 3) (64980, 3)


The other datasets that are provided with ``neuromaps`` are annotations
(i.e., brain maps!). While we are slowly making more and more of these openly
available, for now only a subset are accessible to the general public; these
are returned by default via :func:`datasets.available_annotations`.



In [3]:
annotations = datasets.available_annotations()
print(f'Available annotations: {len(annotations)}')

Available annotations: 86


The :func:`~.available_annotations` function accepts a number of keyword
arguments that you can use to query specific datasets. For example, providing
the `format='volume`' argument will return only those annotations that
are, by default, a volumetric image:



In [4]:
volume_annotations = datasets.available_annotations(format='volume')
print(f'Available volumetric annotations: {len(volume_annotations)}')

Available volumetric annotations: 47


There are a number of keyword arguments we can specify to reduce the scope of
the annotations returned. Here, `source` specifies where the annotation came
from (i.e., a dataset from a manuscript or a data repository or toolbox),
`desc` refers to a brief description of the annotation, `space` clarifies
which space the annotation is in, and `den` (specific to surface annotations)
clarifies the density of the surface on which the annotation is defined:



In [5]:
annot = datasets.available_annotations(source='abagen', desc='genepc1',
                                       space='fsaverage', den='10k')
print(annot)

[('abagen', 'genepc1', 'fsaverage', '10k')]


Annotations also have tags to help sort them into categories. You can see
what tags can be used to query annotations with the :func:`~.available_tags`
functions:



In [6]:
tags = datasets.available_tags()
print(tags)

['ASL', 'MEG', 'MRI', 'PET', 'fMRI', 'functional', 'genetics', 'meta-analysis', 'metabolism', 'receptors', 'resteyesopen', 'structural']


Tags can be used as a keyword argument with :func:`~.available_annotations`.
You can supply either a single tag or a list of tags. Note that supplying a
list will only return those annotations that match ALL supplied tags:



In [7]:
fmri_annotations = datasets.available_annotations(tags='fMRI')
print(fmri_annotations)

[('margulies2016', 'fcgradient01', 'fsLR', '32k'), ('margulies2016', 'fcgradient02', 'fsLR', '32k'), ('margulies2016', 'fcgradient03', 'fsLR', '32k'), ('margulies2016', 'fcgradient04', 'fsLR', '32k'), ('margulies2016', 'fcgradient05', 'fsLR', '32k'), ('margulies2016', 'fcgradient06', 'fsLR', '32k'), ('margulies2016', 'fcgradient07', 'fsLR', '32k'), ('margulies2016', 'fcgradient08', 'fsLR', '32k'), ('margulies2016', 'fcgradient09', 'fsLR', '32k'), ('margulies2016', 'fcgradient10', 'fsLR', '32k'), ('mueller2013', 'intersubjvar', 'fsLR', '164k'), ('neurosynth', 'cogpc1', 'MNI152', '2mm')]


Once we have an annotation that we want we can use the
:func:`neuromaps.datasets.fetch_annotation` to actually download the
files. This has a very similar signature to the
:func:`~.available_annotations` function, accepting almost all the same
keyword arguments to specify which annotations are desired.

Here, we'll grab the first principal component of gene expression across the
brain (from the Allen Human Brain Atlas):



In [8]:
abagen = datasets.fetch_annotation(source='abagen', desc='genepc1')
print(abagen)

[fetch_single_file] Downloading data from 
https://files.osf.io/v1/resources/4mw3a/providers/osfstorage/60c2290118f70b01fca797eb ...

[fetch_single_file]  ...done. (1 seconds, 0 min)

[fetch_single_file] Downloading data from 
https://files.osf.io/v1/resources/4mw3a/providers/osfstorage/60c228f5f3ce9401fa24e521 ...

[fetch_single_file]  ...done. (6 seconds, 0 min)


[References] Please cite the following papers if you are using this data:

  For {'source': 'abagen', 'desc': 'genepc1', 'space': 'fsaverage', 'den': '10k'}:
  [primary]:
    Michael J Hawrylycz, Ed S Lein, Angela L Guillozet-Bongaarts, Elaine H Shen, Lydia Ng, Jeremy A Miller, Louie N Van De Lagemaat, Kimberly A Smith, Amanda Ebbert, Zackery L Riley, and others. An anatomically comprehensive atlas of the adult human brain transcriptome. Nature, 489(7416):391, 2012.
    Ross D Markello, Aurina Arnatkeviciute, Jean-Baptiste Poline, Ben D Fulcher, Alex Fornito, and Bratislav Misic. Standardizing workflows in imaging transcriptomics with the abagen toolbox. eLife, 10:e72129, 2021.
  [secondary]:
    
['/Users/kevinhuang/neuromaps-data/annotations/abagen/genepc1/fsaverage/source-abagen_desc-genepc1_space-fsaverage_den-10k_hemi-L_feature.func.gii', '/Users/kevinhuang/neuromaps-data/annotations/abagen/genepc1/fsaverage/source-abagen_desc-genepc1_space-fsaverage_den-10k_hemi-R_feature.func.g

Notice that the returned annotation ``abagen`` is a dictionary. We can subset
the dictionary with the appropriate key or, if we know that our query is
going to return only one annotation, also provide the `return_single=True`
argument to the fetch call:



In [9]:
abagen = datasets.fetch_annotation(source='abagen', desc='genepc1',
                                   return_single=True)
print(abagen)


[References] Please cite the following papers if you are using this data:

  For {'source': 'abagen', 'desc': 'genepc1', 'space': 'fsaverage', 'den': '10k'}:
  [primary]:
    Michael J Hawrylycz, Ed S Lein, Angela L Guillozet-Bongaarts, Elaine H Shen, Lydia Ng, Jeremy A Miller, Louie N Van De Lagemaat, Kimberly A Smith, Amanda Ebbert, Zackery L Riley, and others. An anatomically comprehensive atlas of the adult human brain transcriptome. Nature, 489(7416):391, 2012.
    Ross D Markello, Aurina Arnatkeviciute, Jean-Baptiste Poline, Ben D Fulcher, Alex Fornito, and Bratislav Misic. Standardizing workflows in imaging transcriptomics with the abagen toolbox. eLife, 10:e72129, 2021.
  [secondary]:
    
['/Users/kevinhuang/neuromaps-data/annotations/abagen/genepc1/fsaverage/source-abagen_desc-genepc1_space-fsaverage_den-10k_hemi-L_feature.func.gii', '/Users/kevinhuang/neuromaps-data/annotations/abagen/genepc1/fsaverage/source-abagen_desc-genepc1_space-fsaverage_den-10k_hemi-R_feature.func.g

And that's it! This example provided a quick overview on how to fetch the
various atlases and datasets provided with ``neuromaps``. For more
information please refer to the `API reference <ref_datasets>`.

